# Evaluation of Univariate Functions over Small Domains in Secure Multi-Party Computation Protocols

Secure multi-party computation (MPC) protocols make it possible for an ensemble of parties to jointly compute the result of a mutually known function on encrypted inputs. Some MPC protocols (such as additive secret sharing and [Shamir's secret sharing](https://en.wikipedia.org/wiki/Shamir%27s_secret_sharing) schemes) rely on secret sharing operations that are homormophic with respect to addition. Thus, such protocols enable (1) addition of two secret-shared values and (2) multiplication of a secret-shared value by an in-the-clear (*i.e.*, known to all parties) scalar value. Such schemes are sometimes called *linear*.

Notably, protocols that rely on linear schemes do not require that the parties communicate during the course of a computation (with the exception of an initial round in which the inputs are separated into shares and a final round in which the output value is reconstructed from shares). In contrast to this, more general-purpose protocols [[1]](#ref1) that allow the evaluation of other functions on secret-shared values usually rely on (1) additional rounds of communication during the course of a computation and/or (2) additional rounds of communication during a preprocessing phase.

This document provides an overview of a particular exception to the above that can be exploited for any linear scheme when the domain of possible inputs is *small*. In particular, it is possible to compute *any* univariate function over secret-shared inputs -- *without* additional rounds of communication -- by leveraging a [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) in which individual values are represented as vectors. The protocol construction approach presented in this document distills a technique that has been employed in earlier related work [[2](#ref2), [3](#ref3)]. The intent of presenting this technique in a formal, isolated form (independently of any particular application) is to facilitate the identification of new applications, variants, and extensions thereof.

## Definitions

Let $n \in \mathbb{N}$ be a natural number such that $n \geq 2$. Assume that addition and scalar multiplication are defined in a componentwise manner on vectors such that $\langle a, b \rangle + \langle c, d \rangle = \langle a + c, b + d \rangle$ and $a \cdot \langle b, c \rangle = \langle a \cdot b, a \cdot c \rangle$.

A *linear secret sharing scheme* consists of a tuple $\langle R, S, \mathsf{shares}, \mathsf{output}\rangle$ where

* $R$ and $S$ are finite rings where $R \cong S$ such that any $a \in R$ has a corresponding $a' \in S$,
* $\mathsf{shares}: \mathbb{N} \times R \to S^n$ is a function that uses a seed value $\sigma \in \mathbb{N}$ to map an element to a vector of $n$ secret shares, and
* $\mathsf{output}: S^n \to R$ is a function that maps a vector of secret shares to a single element.

In such a scheme, the below must be true for any $\sigma \in \mathbb{N}$ and $a \in R$:

$$\mathsf{output}(\mathsf{shares}(\sigma, a)) = a.$$

Also, the $\mathsf{shares}$ and $\mathsf{output}$ maps must represent homomorphisms with respect to addition and scalar multiplication over $R$ and $S^n$. That is, for any $\sigma, \sigma' \in \mathbb{N}$ and $a, b \in R$ (with corresponding $a', b' \in S$), there must exist some $\sigma'', \sigma''' \in \mathbb{N}$ such that:

\begin{align}
\mathsf{shares}(\sigma, a) + \mathsf{shares}(\sigma', b) & = \mathsf{shares}(\sigma'', a + b) \\
a' \cdot \mathsf{shares}(\sigma', b) & = \mathsf{shares}(\sigma''', a \cdot b) \\
\mathsf{output}(\mathsf{shares}(\sigma, a)) + \mathsf{output}(\mathsf{shares}(\sigma', b)) & = \mathsf{output}(\mathsf{shares}(\sigma, a) + \mathsf{shares}(\sigma', b)) \\
a \cdot \mathsf{output}(\mathsf{shares}(\sigma', b)) & = \mathsf{output}(a' \cdot \mathsf{shares}(\sigma', b))
\end{align}

When using elements of $S$ for scalar multiplication, we call them *public* because they must be known to all parties.

While linear secret sharing schemes must also satisfy additional security requirements, these are omitted because they are not consequential for the constructions presented in this document.

## Simple Example

Let $X = \{\textbf{x}, \textbf{y}, \textbf{z}\}$ be a set and let $m: X \to X$ be a map such that:

\begin{align}
m(\textbf{x}) & = \textbf{y} \\
m(\textbf{y}) & = \textbf{y} \\
m(\textbf{z}) & = \textbf{x}
\end{align}

Define a bijection $\varphi: X \to \mathbb{F}_2^3$ such that:

\begin{align}
\varphi(\textbf{x}) & = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix} &
\varphi(\textbf{y}) & = \begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix} &
\varphi(\textbf{z}) & = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}
\end{align}

Consider a matrix $M \in \mathbb{F}_2^{3 \ \times \ 3}$ defined in the manner below:

$$M = \begin{bmatrix} 0 & 0 & 1 \\ 1 & 1 & 0 \\ 0 & 0 & 0 \end{bmatrix}.$$

It is then the case that the transformation represented by $M$ over $\mathbb{F}_2^3$ corresponds to the transformation represented by $m$ over $X$. For example:

$$
M \cdot \varphi(\textbf{z}) = \begin{bmatrix} 0 & 0 & 1 \\ 1 & 1 & 0 \\ 0 & 0 & 0 \end{bmatrix} \cdot \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix} = \begin{bmatrix} 0 \cdot 0 + 0 \cdot 0 + 0 \cdot 1 \\ 1 \cdot 0 + 1 \cdot 0 + 0 \cdot 1 \\ 0 \cdot 0 + 0 \cdot 0 + 0 \cdot 1 \end{bmatrix} = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix} = \varphi(\textbf{x})
$$

Note that it is possible to instantiate a linear secret sharing scheme where $R = S = \mathbb{F}_2$. In that case, the entries of $M \in \mathbb{F}_2^{3 \ \times \ 3}$ can be used as public scalars. Thus, via the homomorphic properties of the secret sharing scheme, the calculation $M \cdot \varphi(\textbf{z})$ would be possible to perform even if $\varphi(\textbf{z})$ were split up into $n$ vectors where each vector represents a secret share of $\varphi(\textbf{z})$.

## Protocol Construction

Let $\langle R, S, \mathsf{shares}, \mathsf{output} \rangle$ be a linear secret sharing scheme. Consider a finite set $X = \{x_1, \ldots, x_k\}$ for $k \in \mathbb{N}$. Define a bijection $\varphi: X \to R^k$ such that for any $x_i \in X$, $\varphi(x_i) = \langle 0, \ldots, 1, \ldots, 0 \rangle$ is a one-hot encoding of $x_i$ where $\varphi(x_i)_i = 1$ and $\varphi(x_i)_j = 0$ for all $j \neq i$.

A secret-sharing scheme (not necessarily linear) can be defined for $R^k$ in a componentwise manner in terms of $\langle R, S, \mathsf{shares}, \mathsf{output} \rangle$. In particular, a new map $\mathsf{shares}^k: \mathbb{N}^k \times R^k \to (S^k)^n$ can be defined in the manner below for any $\langle a_1, \ldots, a_k \rangle \in R^k$ and $\langle \sigma_1, \ldots, \sigma_k \rangle \in \mathbb{N}^k$:

$$
\mathsf{shares}^k (\langle \sigma_1, \ldots, \sigma_k \rangle, \langle a_1, \ldots, a_k \rangle) = \langle \mathsf{shares}(\sigma_1, a_1), \ldots, \mathsf{shares}(\sigma_k, a_k) \rangle.
$$

Likewise, a new map $\mathsf{output}^k: (S^k)^n \to R^k$ can be defined for a collection of vectors of secret shares $\langle v_1, \ldots, v_n \rangle \in (S^k)^n$:

$$
\mathsf{output}^k (\langle v_1, \ldots, v_n \rangle) = \langle \mathsf{output}(\langle (v_1)_1, \ldots, (v_n)_1 \rangle), \ldots, \mathsf{output}(\langle (v_1)_k, \ldots, (v_n)_k \rangle) \rangle.
$$

The maps $\mathsf{shares}^k$ and $\mathsf{output}^k$ can be used to define a secure MPC protocol that supports the evaluation of any univariate function over secret-shared values of $X$.

For any univariate function $m: X \to X$, it is possible to define a matrix $M \in R^{k \ \times \ k}$ where for $i, j \in \{1, \ldots, k\}$:

$$
M_{ij} =
\begin{cases}
1 \ \mathrm{if} \ m(x_j) = x_i\\
0 \ \mathrm{otherwise}
\end{cases}
$$

Given the above definition, $M$ can be viewed as a collection of column vectors in which the $j$th column corresponds to the output $m(x_j)$ for the input $a_j$:

$$M = \begin{bmatrix} \uparrow &  & \uparrow \\ \varphi(m(x_1)) & \ldots & \varphi(m(x_k)) \\ \downarrow &  & \downarrow \end{bmatrix}.$$

Then it is the case that for all $x_i \in X$,

$$M \cdot \varphi(x_i) = \varphi(m(x_i)).$$

Thus, multiplication by $M$ effectively implements the application of the map $m$.

Consider any $x_i \in X$ and a vector $\overline{\sigma} \in \mathbb{N}^k$. Suppose secret shares of $\varphi(x_i)$ are obtained via $\mathsf{shares}^k$:

$$\langle v_1, \ldots, v_n \rangle = \mathsf{shares}^k(\overline{\sigma}, \varphi(x_i)).$$

Because $R \cong S$, there exists a matrix $M' \in S^{k \ \times \ k}$ with entries that correspond to the entries of $M$. Thus, it is possible to compute both $\langle M' \cdot v_1, \ldots, M' \cdot v_n \rangle \in (S^k)^n$ and $\mathsf{output}^k(\langle M' \cdot v_1, \ldots, M' \cdot v_n \rangle) \in R^k$. Furthermore, the homomorphic properties of the underlying scheme $\langle R, S, \mathsf{shares}, \mathsf{output} \rangle$ imply that

$$\mathsf{output}^k(\langle M' \cdot v_1, \ldots, M' \cdot v_n \rangle) = M \cdot \varphi(x_i).$$

Thus, the protocol enables the calculation of $M \cdot \varphi(x_i)$ on a secret-shared version of $\varphi(x_i)$. Furthermore, the associativity of matrix multiplication implies that any number of univariate functions can be applied in sequence to a secret-shared version of an initial input $\varphi(x_i)$.

## Communication Cost

Representing inputs using a one-hot encoding can significantly increase communication costs when distributing shares and when reconstructing the result. However, the presented protocol may still be an improvement over alternatives when these costs are amortized over the course of a computation. In certain circumstances, the reduction in communication cost may have no upper bound over alternatives (such as in a scenario in which a secret-shared state of a state transition machine is updated an indefinite number of times using a univariate update function).

As an example, consider a scenario in which $n = 8$, $R = S = \mathbb{F}_2$, and $X = \mathbb{F}_{256}$. An $8$-bit vector would be sufficient to represent an element in $X$ within an additive secret-sharing scheme, while $256 = 8 \cdot 32$ bits would be required to represent an element in $R^{256}$. This factor of $32$ is *independent* of the number of univariate functions applied to the secret-shared input. By contrast, the cost of an additional round of communication in an alternative protocol may be determined by both the number of non-linear operations and the number of participating parties.

Continuing with the same example, suppose that the scenario involves an outsourced MPC computation. Thus, a data contributor using the protocol presented in this work must send $(8 \cdot 32) \cdot 8$ bits in total. In an alternative protocol that does not rely on a one-hot encoding, only $8 \cdot 8$ bits would need to be sent. Suppose that in the alternative protocol, any additional round of communication requires each party to send one element from $\mathbb{F}_{256}$ to every other party. This requires $8 \cdot (8 \cdot 7)$ bits to be sent during each additional round. After five such additional rounds, the total amount of data that must be sent in the alternative protocol exceeds the amount of data that must be sent in the presented protocol: $(8 \cdot 8) + 5 \cdot (8 \cdot (8 \cdot 7)) > (8 \cdot 32) \cdot 8$ (*i.e.*, $36 \cdot 8^2 > 32 \cdot 8^2$).

## Protocol Variants and Applications

The protocol construction presented in this document can be extended or modified in a variety of ways. A few such variants are discussed in this section.

**Output Selection**. While a one-hot encoding introduces significant overheads, it may be possible in some implementations to avoid these overheads during the final round of communication. In particular, the secret-shared one-hot encoding can be used to *select* an output *before* the final round if $R = X$ (a simplified variant of techniques used in related earlier work [[2]](#ref2)). For example, suppose $R = S = X = \mathbb{F}_4$. Then a one-hot encoding of $2 \in X$ might be $\langle 0, 0, 1, 0 \rangle \in R^4$. Performing the multiplication of vectors below would yield the *single* element $2 \in X$ directly:

$$\begin{bmatrix} 0 & 1 & 2 & 3 \end{bmatrix} \cdot \begin{bmatrix} 0 \\ 0 \\ 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 2 \end{bmatrix}.$$

Within the context of the MPC protocol, the above computation would be performed using a public vector $\langle 0, 1, 2, 3\rangle \in S^4$ and a secret-shared version of $\langle 0, 0, 1, 0 \rangle \in R^4$. The secret shares of the result $\langle 2 \rangle$ could be then be used to obtain the output as a single value, reducing the amount of information sent by a factor of four.

**Secret-Shared Functions.** For any $v \in S^k$, the individual entries in the vector $M' \cdot v$ are sums of secret-shared values, each of which is a product of a public scalar (*i.e.*, some entry $M_{ij}$) and a secret-shared value (*i.e.*, a component of the vector $v$). It is worth noting that the same would be true if the entries of the matrix were secret-shared values and the components of $v$ were public scalars. Thus, the above approach can be adapted to a scenario in which a secret-shared *function* is applied to a public input vector. In particular, secret-shared entries of a matrix $M$ would be distributed to the parties and each party would apply its matrix of secret shares to the same agreed-upon public input.

**Secret Permutations and Shuffles**. Any permutation of an ordered collection of items can be represented as a map. Thus, the presented protocol can be used to permute secret-shared data using a public permutation or (via the variant discussed above in which a secret-shared function is applied) to permute public data via a secret-shared permutation. In the latter case, the result can be viewed as a secret *shuffle* [[4]](#ref4), albeit of *public* input data.

## References

1. <a id='ref1'></a>Beaver, D. [Efficient Multiparty Protocols Using Circuit Randomization](https://dl.acm.org/doi/10.5555/646756.705383). *CRYPTO '91*, 420–432, 1991.

2. <a id='ref2'></a>Launchbury, J., Diatchki, I.S., DuBuisson, T., Adams-Moran, A. [Efficient lookup-table protocol in secure multiparty computation](https://dl.acm.org/doi/10.1145/2398856.2364556). *ACM SIGPLAN Notices*, **47**(9):189–200, 2012.

3. <a id='ref3'></a>Abidin, A., Aly, A., Cleemput, S., Mustafa, M.A. [An MPC-Based Privacy-Preserving Protocol for a Local Electricity Trading Market](https://link.springer.com/chapter/10.1007/978-3-319-48965-0_40). *Cryptology and Network Security*, **10052**:615–625, 2016.

4. <a id='ref4'></a>Chida, K., Hamada, K., Ikarashi, D., Kikuchi, R., Kiribuchi, N., Pinkas, B. [An
efficient secure three-party sorting protocol with an honest majority](https://eprint.iacr.org/2019/695). IACR 
Cryptology ePrint Archive, 2019/695, 2019.